# Análise Inicial de Transações - Projeto AML/Fraud
Este notebook realiza a exploração inicial do dataset de transações simuladas.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Carregar os dados
file_path = '/mnt/data/aml_fraud_risk_simulated_v1.csv'
df = pd.read_csv(file_path)
df.head()

## Informações Gerais
Aqui analisamos dimensões, tipos de dados e valores ausentes.

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.isnull().sum()

## Distribuição de Variáveis
Visualizações para entender a distribuição de valores.

In [ ]:
plt.figure(figsize=(10, 6))
df['amount'].hist(bins=50)
plt.title('Distribuição de Valores de Transações')
plt.xlabel('Valor (USD)')
plt.ylabel('Frequência')
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.countplot(data=df, x='origin_country', order=df['origin_country'].value_counts().index)
plt.title('Origem das Transações')
plt.ylabel('Contagem')
plt.grid(True)
plt.show()

## Correlação entre variáveis numéricas

In [ ]:
plt.figure(figsize=(10, 6))
sns.heatmap(df.corr(numeric_only=True), annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Matriz de Correlação')
plt.show()

### Próximos passos:
- Criar score manual de risco com base em regras simples
- Gerar coluna `manual_risk_score`
- Analisar comportamento de alto risco